In [3]:
from py2neo import Graph, Node, Relationship
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph

import os

load_dotenv()

# Connect to the Neo4j database

NEO4J_GRAPH_URL = os.getenv("NEO4J_GRAPH_URL") 
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME") 
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD") 
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE") 

graph = Neo4jGraph(url=NEO4J_GRAPH_URL, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

In [5]:
graph.get_schema

'Node properties:\nPeraturan {judul: STRING, tanggal_berlaku: LOCAL_DATE_TIME, nomor_ketentuan: STRING, jenis_bank: STRING, id: STRING, jenis_ketentuan: STRING}\nRelationship properties:\n\nThe relationships:\n(:Peraturan)-[:MENCABUT]->(:Peraturan)\n(:Peraturan)-[:DIUBAH]->(:Peraturan)\n(:Peraturan)-[:MENGUBAH]->(:Peraturan)\n(:Peraturan)-[:DICABUT]->(:Peraturan)'

In [13]:
from py2neo import Graph, Node, Relationship

# Connect to the Neo4j database
graph = Graph(NEO4J_GRAPH_URL, name=NEO4J_USERNAME, password=NEO4J_PASSWORD)

476

In [8]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator

class Peraturan(BaseModel):
    mencabut: str = Field(description="Nomor Peraturan yang mencabut")
    dicabut: list[str] = Field(description="Semua Nomor Peraturan yang dicabut secara Keseluruhan dan bukan beberapa pasal atau ayat tertentu saja")


In [9]:
import pickle

with open('result_ojk_ls.pkl', 'rb') as f:
    data = pickle.load(f)

In [10]:
data

[Peraturan(mencabut='10/15/PBI/2008', dicabut=['3/21/PBI/2001', '8/6/PBI/2006', '9/13/PBI/2007', '26/1/BPPP']),
 Peraturan(mencabut='10/17/PBI/2008', dicabut=['6/24/PBI/2004', '8/25/PBI/2006', '8/9/DPbS']),
 Peraturan(mencabut='10/18/PBI/2008', dicabut=['31/150/KEP/DIR', '2/15/PBI/2000', '8/21/PBI/2006', '9/9/PBI/2007', '8/24/PBI/2006']),
 Peraturan(mencabut='10/19/PBI/2008', dicabut=['6/15/PBI/2004', '7/49/PBI/2005']),
 Peraturan(mencabut='10/POJK.03/2016', dicabut=['6/27/PBI/2004']),
 Peraturan(mencabut='11/15/PBI/2009', dicabut=['8/3/PBI/2006', '9/7/PBI/2007']),
 Peraturan(mencabut='11/1/PBI/2009', dicabut=['11/30 UM/PU', '23/3/BPPB', '2/27/PBI/2000']),
 Peraturan(mencabut='11/24/PBI/2009', dicabut=['5/3/PBI/2003', '7/23/PBI/2005']),
 Peraturan(mencabut='11/28/PBI/2009', dicabut=['3/10/PBI/2001', '5/21/PBI/2003']),
 Peraturan(mencabut='11/3/PBI/2009', dicabut=['6/24/PBI/2004', '7/35/PBI/2005']),
 Peraturan(mencabut='11/POJK.03/2016', dicabut=['15/12/PBI/2013', '9/31/DPNP']),
 Peratu

In [21]:
with open('result_bi.pkl', 'rb') as f:
    data_bi = pickle.load(f)

In [17]:
dc = []
for peraturan in data:
    no = peraturan.mencabut
    ls_dicabut = peraturan.dicabut
    for dicabut in ls_dicabut:
        dc.append({"Nomor Ketentuan": no, "Mencabut": dicabut})

In [23]:
for peraturan in data_bi:
    no = peraturan.mencabut
    ls_dicabut = peraturan.dicabut
    for dicabut in ls_dicabut:
        dc.append({"Nomor Ketentuan": no, "Mencabut": dicabut})

In [24]:
import pandas as pd

df = pd.DataFrame(dc)
df

,Nomor Ketentuan,Mencabut
0,10/15/PBI/2008,3/21/PBI/2001
1,10/15/PBI/2008,8/6/PBI/2006
2,10/15/PBI/2008,9/13/PBI/2007
3,10/15/PBI/2008,26/1/BPPP
4,10/17/PBI/2008,6/24/PBI/2004
...,...,...
1034,9/13/PBI/2007,5/12/PBI/2003
1035,9/13/PBI/2007,5/12/PBI/2003
1036,9/14/PBI/2007,7/8/PBI/2005
1037,9/19/PBI/2007,7/46/PBI/2005


In [84]:
# Create nodes

new_node_counter = 0
for index, row in df.iterrows():
    nomor_ketentuan = row['Nomor Ketentuan']
    existence = graph.query(f"MATCH (p:Peraturan {{nomor_ketentuan: '{nomor_ketentuan}'}}) RETURN p")
    existence = existence.to_ndarray()
    if len(existence) == 0:
        node = Node("Peraturan", id=nomor_ketentuan, nomor_ketentuan=nomor_ketentuan)
        graph.merge(node, "Peraturan", "id")
        new_node_counter += 1
        print("Sucessfully create node for", nomor_ketentuan)
        

Sucessfully create node for 10/15/PBI/2008
Sucessfully create node for 10/19/PBI/2008
Sucessfully create node for 11/24/PBI/2009
Sucessfully create node for 11/28/PBI/2009
Sucessfully create node for 12/19/PBI/2010
Sucessfully create node for 14/16/PBI/2012
Sucessfully create node for 14/18/PBI/2012
Sucessfully create node for 14/22/PBI/2012
Sucessfully create node for 6/17/PBI/2004
Sucessfully create node for 6/22/PBI/2004
Sucessfully create node for 6/23/PBI/2004
Sucessfully create node for 7/10/PBI/2005
Sucessfully create node for 7/1/PBI/2005
Sucessfully create node for 7/2/PBI/2005
Sucessfully create node for 7/8/PBI/2005
Sucessfully create node for 8/20/PBI/2006
Sucessfully create node for 8/21/PBI/2006
Sucessfully create node for 9/13/PBI/2007
Sucessfully create node for PER-11/BL/2012
Sucessfully create node for 199/PMK.010/2008
Sucessfully create node for 53/PMK.010/2012
Sucessfully create node for 79/PMK.010/2009
Sucessfully create node for 10/POJK.05/2019
Sucessfully create 

In [85]:
print(new_node_counter)

259


In [101]:
type(node)

py2neo.data.Node

In [98]:
node_query = graph.query(f"MATCH (p:Peraturan {{nomor_ketentuan: '11/1/PBI/2009'}}) RETURN p").data()[0]['p']

In [99]:
type(node_query)

py2neo.data.Node

In [102]:
# Function to create relationships
def create_relationship(source_id, target_id, relationship_type):
    existence = graph.query(f"MATCH (p:Peraturan {{nomor_ketentuan: '{target_id}'}}) RETURN p")
    existence = existence.to_ndarray()
    if len(existence) == 0:
        node = Node("Peraturan", id=target_id, nomor_ketentuan=target_id)
        graph.merge(node, "Peraturan", "id")
        print("Sucessfully create node for", target_id)
        
    source_node = graph.query(f"MATCH (p:Peraturan {{nomor_ketentuan: '{source_id}'}}) RETURN p").data()[0]['p']
    target_node = graph.query(f"MATCH (p:Peraturan {{nomor_ketentuan: '{target_id}'}}) RETURN p").data()[0]['p']
    relationship = Relationship(source_node, relationship_type, target_node)
    graph.merge(relationship)
    print("Sucessfully create", source_id, relationship, target_id)


# Create relationships based on columns
for index, row in df.iterrows():
    source_id = row['Nomor Ketentuan']
    create_relationship(source_id, row['Mencabut'], "MENCABUT")
    create_relationship(row['Mencabut'], source_id, "DICABUT")


Sucessfully create node for 3/21/PBI/2001
Sucessfully create 10/15/PBI/2008 (_476)-[:MENCABUT {}]->(_735) 3/21/PBI/2001
Sucessfully create 3/21/PBI/2001 (_735)-[:DICABUT {}]->(_476) 10/15/PBI/2008
Sucessfully create 10/15/PBI/2008 (_476)-[:MENCABUT {}]->(_328) 8/6/PBI/2006
Sucessfully create 8/6/PBI/2006 (_328)-[:DICABUT {}]->(_476) 10/15/PBI/2008
Sucessfully create 10/15/PBI/2008 (_476)-[:MENCABUT {}]->(_493) 9/13/PBI/2007
Sucessfully create 9/13/PBI/2007 (_493)-[:DICABUT {}]->(_476) 10/15/PBI/2008
Sucessfully create node for 26/1/BPPP
Sucessfully create 10/15/PBI/2008 (_476)-[:MENCABUT {}]->(_736) 26/1/BPPP
Sucessfully create 26/1/BPPP (_736)-[:DICABUT {}]->(_476) 10/15/PBI/2008
Sucessfully create 10/17/PBI/2008 (_401)-[:MENCABUT {}]->(_392) 6/24/PBI/2004
Sucessfully create 6/24/PBI/2004 (_392)-[:DICABUT {}]->(_401) 10/17/PBI/2008
Sucessfully create node for 8/25/PBI/2006
Sucessfully create 10/17/PBI/2008 (_401)-[:MENCABUT {}]->(_737) 8/25/PBI/2006
Sucessfully create 8/25/PBI/2006 (_

In [74]:
nomor_ = "12/POJK.03/2021"

In [81]:
res = graph.query(f"MATCH (p:Peraturan {{nomor_ketentuan: '{nomor_}'}}) RETURN p")
res

p
"(_21:Peraturan {id: '12/POJK.03/2021', jenis_bank: 'Bank Umum Konvensional', jenis_ketentuan: 'POJK', judul: 'Bank Umum', nomor_ketentuan: '12/POJK.03/2021', tanggal_berlaku: datetime('2021-07-30T00:00:00.000000000')})"


In [57]:
res_2 = graph.query("MATCH (p:Peraturan {nomor_ketentuan: '11'}) RETURN p")
res_2

(No data)

In [76]:
res = res.to_ndarray()

In [59]:
res_2 = res_2.to_ndarray()

In [77]:
res

array([[Node('Peraturan', id='12/POJK.03/2021', jenis_bank='Bank Umum Konvensional', jenis_ketentuan='POJK', judul='Bank Umum', nomor_ketentuan='12/POJK.03/2021', tanggal_berlaku=DateTime(2021, 7, 30, 0, 0, 0.0))]],
      dtype=object)

In [61]:
res_2

array([], dtype=float64)

In [78]:
len(res)

1

In [83]:
if len(res.to_ndarray()):
    print(True)

In [105]:
len(graph.nodes)

1265